# Plot composite profile

## Import package

In [1]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from scipy.stats import percentileofscore

## Load data

In [2]:
REG_PATH = "/work/b11209013/2025_Research/regression/"

# load ERA5 regression
with h5py.File(f"{REG_PATH}IMERG_ERA5.h5", "r") as f:
    q_grp = f["q_composite"]
    t_grp = f["t_composite"]
    
    q_reg = {}
    t_reg = {}

    for key in q_grp.keys():
        q_reg[key] = np.array(q_grp.get(key))
        t_reg[key] = np.array(t_grp.get(key))

# load CloudSat regression
with h5py.File(f"{REG_PATH}IMERG_CLOUDSAT.h5", "r") as f:
    lw_grp = f["lw_composite"]
    sw_grp = f["sw_composite"]
    
    lw_reg = {}
    sw_reg = {}

    for key in lw_grp.keys():
        lw_reg[key] = np.array(lw_grp.get(key))
        sw_reg[key] = np.array(sw_grp.get(key))

# load ERA5 radiation regression
with h5py.File(f"{REG_PATH}IMERG_MOISTRAD.h5", "r") as f:
    q_lw_grp = f["q_lw_composite"]
    q_sw_grp = f["q_sw_composite"]
    t_lw_grp = f["t_lw_composite"]
    t_sw_grp = f["t_sw_composite"]
    
    q_lw_reg = {}
    q_sw_reg = {}
    t_lw_reg = {}
    t_sw_reg = {}

    for key in q_lw_grp.keys():
        q_lw_reg[key] = np.array(q_lw_grp.get(key))
        q_sw_reg[key] = np.array(q_sw_grp.get(key))
        t_lw_reg[key] = np.array(t_lw_grp.get(key))
        t_sw_reg[key] = np.array(t_sw_grp.get(key))

# load ERA5 Q1 regression
with h5py.File(f"{REG_PATH}IMERG_Q1.h5", "r") as f:
    Q1 = {}

    for key in f.keys():
        Q1[key] = np.array(f.get(key))

PROFILES: dict[str, dict[str, np.ndarray]] = {
    "LW_total": lw_reg,
    "SW_total": sw_reg,
    "LW_moist": q_lw_reg,
    "SW_moist": q_sw_reg,
    "LW_temp" : t_lw_reg,
    "SW_temp" : t_sw_reg,
    "Q1"      : Q1
}


### Compute Cloud Only Component

In [3]:
PROFILES["LW_cld"] = {
    key: PROFILES["LW_total"][key] - PROFILES["LW_moist"][key] - PROFILES["LW_temp"][key]
    for key in PROFILES["LW_total"].keys()
}

PROFILES["SW_cld"] = {
    key: PROFILES["SW_total"][key] - PROFILES["SW_moist"][key] - PROFILES["SW_temp"][key]
    for key in PROFILES["SW_total"].keys()
}

### Compute vertical profile at convective phase

#### Define functions

In [4]:
def parse_key(key: str):
    hov_type, kl, kr = key.split("_")
    return hov_type, kl, kr 

def phase_avg(
    data, central_idx, key
):

    # calculate size of averaging domain
    separate_key = parse_key(key)

    k_ave = (int(separate_key[1]) + int(separate_key[2]))/2

    kernel_lens = np.round(2*np.pi*6.371*1e6*1e-3/(k_ave*4*110))/0.625
    half_kernel = int(kernel_lens // 2)

    # slice of window indexing
    sel_idx = slice(central_idx-half_kernel, central_idx+half_kernel+1)

    # select radiative heating profile
    data_comp = data[:,sel_idx]
    
    # average over slected domain
    data_mean = np.nanmean(data_comp,axis=-1)

    return data_mean

#### Compute for Total Radiation

In [5]:
MEAN_PROFILES = {}

for key, out_dict in PROFILES.items():
    MEAN_PROFILES[key] = {
        key: phase_avg(data = out_dict[key], central_idx = 288, key=key)
        for key in out_dict.keys()
    }

### Calculate error bar range

In [6]:
# Define standard errors
def standard_error(
        prof_dict: dict[str, np.ndarray]
) -> dict[str, np.ndarray]:
    return {
        key: value.std(axis=0, ddof=1)/np.sqrt(value.shape[0])
        for key, value in prof_dict.items()
    }

SE_PROFILES = {}

for key in MEAN_PROFILES.keys():
    SE_PROFILES[key] = standard_error(MEAN_PROFILES[key])


## Plot out composite profile

### Define plot functions

#### Plot x-z composite

In [7]:
def plot_profile(
        lon, lev, rad_data,
        t_data, q_data,
        cmap, key, type_of_rad
):
    # split key
    split_key = key.split("_")
    hov_type = split_key[0]
    kl = split_key[1]
    kr = split_key[2]

    split_type = type_of_rad.split("_")
    type_of_rad = split_type[0]
    type_of_comp = split_type[1]

    # Calculate levels
    if (type_of_comp == "Total") or (type_of_comp == "Cloud"):
        rad_max   = 0.1
    else:
        rad_max   = 0.0005
    
    if (type_of_comp == "Total") and (type_of_rad == "Q1"):
        rad_max = 0.5

    rad_max = np.nanmax(rad_data) *0.95

    t_max     = np.abs(t_data).max() // 0.001
    t_levels  = np.linspace(-t_max*0.001, t_max*0.001+0.001, 15)
    t_levels  = [t_levels[i] for i in range(len(t_levels)) if abs(t_levels[i]) >= 1e-3]
    
    q_max     = np.abs(q_data).max() // 0.001
    q_levels  = np.linspace(-q_max*0.001, q_max*0.001+0.001, 15)
    q_levels  = [q_levels[i] for i in range(len(q_levels)) if abs(q_levels[i]) >= 1e-3]

    fig = plt.figure(figsize=(10.5,6.2))

    pc = plt.pcolormesh(
        lon, lev, rad_data,
        cmap=cmap, norm=TwoSlopeNorm(vmin=-rad_max, vmax=rad_max, vcenter=0),
        alpha=0.5
        )

    ct = plt.contour(lon, lev, t_data, colors='black', linewidths=1.25, levels=t_levels)
    plt.clabel(ct, fmt='%.3f', colors='black', fontsize=8)

    ct = plt.contour(lon, lev, q_data, colors='seagreen', linewidths=1.25, levels=q_levels)
    plt.clabel(ct, fmt='%.3f', colors='seagreen', fontsize=8)

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.xlim(-100, 100)
    plt.xlabel("Longitude", fontsize=18)
    plt.ylabel("Level (hPa)", fontsize=18)
    plt.title(f"{hov_type.upper()} k={kl}~{kr} {type_of_rad} ({type_of_comp})", fontsize=20)
    plt.gca().invert_yaxis()
    cbar = plt.colorbar(
        pc, label=f"{type_of_rad} Regression Coefficient",
        orientation="horizontal", shrink=0.8, aspect=50
        )
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.xaxis.label.set_size(16)
    plt.tight_layout()
    plt.savefig(f"/home/b11209013/2025_Research/Obs/Figure/{type_of_comp}_Rad/{type_of_rad}/{hov_type}_k={kl}_{kr}.png", dpi=500)
    plt.close()

#### Plot Convective Phase

In [8]:
def plot_convphase(
        lev, prof, err, type_of_rad
        ):
    
    split_type_of_rad = type_of_rad.split("_")
    rad = split_type_of_rad[0]
    comp = split_type_of_rad[1]

    if type_of_rad == "LW_Moisture" or type_of_rad == "SW_Moisture":
        xtick = np.linspace(-0.009, 0.009, 5)
    
    elif type_of_rad == "Q1_Total":
        xtick = np.linspace(-0.5, 0.5, 5)

    else:
        xtick = np.linspace(-0.1, 0.1, 5)

    cmap  = plt.get_cmap("jet")
    clist = [cmap(i/6) for i in range(len(prof.keys()))]

    fig, ax = plt.subplots(figsize=(9, 15))

    for i, key in enumerate(prof.keys()):
        ax.plot(
            prof[key], lev,
            label=key, color=clist[i]
        )
        ax.errorbar(prof[key], lev, xerr=err[key], fmt="none",
                    color=clist[i], lw=1, capsize=2
                    )
        
    ax.set_xticks(xtick)
    ax.set_xticklabels([f"{v:.4f}" for v in xtick], fontsize=18)
    ax.set_yticks(np.linspace(1000, 100, 10))
    ax.set_yticklabels(["1000","900","800","700","600","500","400","300","200","100"], fontsize=18)
    ax.set_xlim(xtick[0]*1.01, xtick[-1]*1.01)
    ax.set_ylim(1000, 100)
    ax.legend(fontsize=18)
    ax.grid(True)
    ax.set_xlabel(f"Averaged {rad.upper()} Heating", fontsize=20)
    ax.set_ylabel("Pressure (hPa)", fontsize=20)
    # ax.set_title(f"Averaged {rad.upper()} {comp.upper()} Heating Profiles", fontsize=20)
    plt.tight_layout()
    plt.savefig(f"/home/b11209013/2025_Research/Obs/Figure/ConvPhase/{rad}_{comp}.png", dpi=500)
    plt.close()

### Plot relative ratio

In [9]:
def plot_convphase_rel(
        lev, prof, type_of_rad
        ):
    
    split_type_of_rad = type_of_rad.split("_")
    rad = split_type_of_rad[0]
    comp = split_type_of_rad[1]

    cmap  = plt.get_cmap("jet")
    clist = [cmap(i/6) for i in range(len(prof.keys()))]

    fig, ax = plt.subplots(figsize=(9, 15))

    for i, key in enumerate(prof.keys()):
        ax.plot(
            prof[key]/prof[key].max(), lev,
            label=key, color=clist[i]
        )
        
    ax.set_xticks(np.linspace(-1, 1, 5))
    ax.set_xticklabels([f"{v:.4f}" for v in np.linspace(-1, 1, 5)], fontsize=18)
    ax.set_yticks(np.linspace(1000, 100, 10))
    ax.set_yticklabels(["1000","900","800","700","600","500","400","300","200","100"], fontsize=18)
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(1000, 100)
    ax.legend(fontsize=18)
    ax.grid(True)
    ax.set_xlabel(f"Averaged {rad.upper()} Heating", fontsize=20)
    ax.set_ylabel("Pressure (hPa)", fontsize=20)
    # ax.set_title(f"Averaged {rad.upper()} {comp.upper()} Heating Profiles", fontsize=20)
    plt.tight_layout()
    plt.savefig(f"/home/b11209013/2025_Research/Obs/Figure/ConvPhase/{rad}_{comp}_rel.png", dpi=500)
    plt.close()

### Plot x-z profile

In [10]:
lon = np.linspace(-180, 180, 576)
lev = np.linspace(1000, 100, 37)
# 'LW_total', 'SW_total', 'LW_moist', 'SW_moist', 'LW_temp', 'SW_temp', 'Q1', 'LW_cld', 'SW_cld'
PROFILE_PLOTS = [
    ("LW_total", "RdBu_r", "LW_Total"),
    ("SW_total", "PiYG_r", "SW_Total"),
    ("LW_moist", "RdBu_r", "LW_Moisture"),
    ("SW_moist", "PiYG_r", "SW_Moisture"),
    ("LW_cld", "RdBu_r", "LW_Cloud"),
    ("SW_cld", "PiYG_r", "SW_Cloud"),
    ("Q1", "RdGy_r", "Q1_Total"),
]

for name, cmap, label in PROFILE_PLOTS:
    for key in PROFILES[name].keys():
        plot_profile(
            lon,
            lev,
            PROFILES[name][key],
            t_reg[key],
            q_reg[key],
            cmap,
            key,
            label
        )

### Plot convective phase profile barplot

In [11]:
for name, cmap, label in PROFILE_PLOTS:
    plot_convphase(
        lev,
        MEAN_PROFILES[name],
        SE_PROFILES[name],
        label
    )

### Plot relative ratio profile

In [12]:
for name, cmap, label in PROFILE_PLOTS:
    plot_convphase_rel(
        lev,
        MEAN_PROFILES[name],
        label
    )